### 一. 基本环境初始化

#### 1. 初始化基本环境

In [1]:
!pwd

/home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy


In [37]:
%%bash
sudo chmod +x init-env.sh
./init-env.sh

### 二. 自定义镜像及Snapshot制备 (可选)

#### 1. 镜像制作(~15min)

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/tools')
cmd = "/bin/sh ./build_and_push.sh %s" %(region_name)
os.system(cmd)

上述命令执行完成后,请检查ECR中是否有对应的存储库和镜像

#### 2. Snapshot 制作(可选)(~20min)

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
Infer_API_IMG= "%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/inference-api:latest" %(account_id,region_name)
Queue_Agent_IMG= "%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/queue-agent:latest" %(account_id,region_name)
OCI_URL = "oci://%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/charts/" %(account_id,region_name)
print ("OCI_URL is :",OCI_URL)
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/utils/bottlerocket-images-cache')
cmd = "/bin/sh ./snapshot.sh %s %s" %(Infer_API_IMG,Queue_Agent_IMG)
os.system(cmd)

In [ ]:
#记录上面输出的snapshot id

### 三. CDK 部署

In [2]:
# 创建模型S3桶
# 替换<bucket name>为自定义桶名，替换<deployment region>为部署区域如'us-east-1'或者'us-west-2'
!aws s3api create-bucket --bucket <bucket_name> --region <deployment_region> 

修改stable-diffusion-on-eks/config.yaml。其中modelBucketArn: 'arn:aws:s3:::<bucket_name>' <bucket_name>为上一步自定义桶名。

其他项请参考文档说明。

最简配置请将：https://github.com/aws-samples/stable-diffusion-on-eks/blob/main/examples/dynamic-runtime.yaml 替换stable-diffusion-on-eks/config.yaml。



#### 执行CDK 部署 (~20min)

In [38]:
import os
os.chdir('/home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/')
cmd = "/bin/sh ./cdk_deploy.sh"
os.system(cmd)

### 四. 模型下载同步
#### 1. 模型下载 (根据模型大小,所需时间也不同)
找到tools目录下S3uploader文件夹,修改其中的down.csv 
修改模型下载url,和目标key列(严格按照prefix/key的结构)后保存

其中prefix 允许值:"stable-diffusion","lora","vae","embeddings"

In [28]:
%%bash
export s3uri=<bucket_name>
cd /home/ec2-user/SageMaker/stable-diffusion-on-eks/deploy/tools/S3uploader
sed -i "s/\,\,/\,$s3uri,/g" down.csv
python3 s3uploader.py down.csv

#### 2. 模型同步
进入datasync 服务访问控制台中找到历史记录是否有执行记录.

### 五.请求测试

In [6]:
#等待cloudformation 执行完成
!aws cloudformation describe-stacks --stack-name SdOnEKSStack --output json --query 'Stacks[0].StackStatus'

"CREATE_COMPLETE"


执行完毕后从cloudformation输出记录 APIGateway 地址

In [33]:
APIGW_URL="$(aws cloudformation describe-stacks --stack-name SdOnEKSStack --output text --query 'Stacks[0].Outputs[?OutputKey==`FrontApiEndpoint`].OutputValue')"
!echo "APIGW URL:" $APIGW_URL


执行完毕后如果需要管理EKS,则执行以下命令,输出记录 eks config update命令,拷贝到bash中执行

In [31]:
!aws cloudformation describe-stacks --stack-name SdOnEKSStack --output text --query 'Stacks[0].Outputs[?OutputKey==`GetAPIKeyCommand`].OutputValue'
# 上一步输出value运行
!aws apigateway get-api-keys --query 'items[?id==``].value' --include-values --output text


In [36]:
!aws cloudformation describe-stacks --stack-name SdOnEKSStack --output text --query 'Stacks[0].Outputs[?OutputKey==`ConfigCommand`].OutputValue'

### 六. Lab2 curl范例

In [32]:
!curl -X POST https://12345678.execute-api.us-east-2.amazonaws.com/prod/ \
    -H 'Content-Type: application/json' \
    -H 'x-api-key: ' \
    -d @test.json
